inspired by: [ModuleForKeithley](https://gist.github.com/rinitha/0844a61a82006fe92c78)

In [9]:
import socket

In [10]:
dmm_ip = "192.168.10.117"
#dmm_ip = "192.168.12.134"

dmm_port = 3490

try:
    dmm_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    dmm_sock.connect((dmm_ip, dmm_port))
    
except Exception as e: 
    print("Something's wrong with %s:%d. Exception is %s" % (dmm_ip, dmm_port, e))

In [11]:
# set timeout on blocking socket operations in [s]
dmm_sock.settimeout(0.5)

In [12]:
scpi_msg = "*IDN?\n"
dmm_sock.sendall(scpi_msg.encode('utf-8'))

In [13]:
reading = dmm_sock.recv(128) # buffer size is 128 bytes
print(reading.decode().strip())

FLUKE,8846A,3729018,08/02/10-11:53


In [13]:
dmm_sock.close()

In [14]:
scpi_msg = "*RST\n"
dmm_sock.sendall(scpi_msg.encode('utf-8'))

In [15]:
scpi_msg = "SYST:REM\n"
dmm_sock.sendall(scpi_msg.encode('utf-8'))

In [9]:
scpi_msg = "CONF:RES DEF\n"
dmm_sock.sendall(scpi_msg.encode('utf-8'))

In [10]:
scpi_msg = "READ?\n"
dmm_sock.sendall(scpi_msg.encode('utf-8'))

In [11]:
reading = dmm_sock.recv(64) # buffer size is 64 bytes
print(reading.decode().strip())

+2.16531600E+02


In [12]:
res_flt = float(reading.decode().strip())

print("Resistor: {:.2f} Ohm".format(res_flt))

Resistor: 216.53 Ohm


In [15]:
scpi_msg = 'FUNC1 "TEMP:FRTD"; FUNC2 "FRES"\n'
dmm_sock.sendall(scpi_msg.encode('utf-8'))

In [33]:
scpi_msg = "READ?;FETCH2?\n"
dmm_sock.sendall(scpi_msg.encode('utf-8'))

In [34]:
reading = dmm_sock.recv(64) # buffer size is 64 bytes

In [35]:
ret_val_list = reading.decode().strip().split(';')

In [36]:
for idx, val in enumerate(ret_val_list):
    ret_val_list[idx] = float(ret_val_list[idx])

In [37]:
print("{:.7f}, {:.7f}".format(ret_val_list[0], ret_val_list[1]))

22.2445000, 108.6652000


# Using the wrapper class 'Fluke_8846A' manually

The new wrapper class **Fluke_8846A** in the python file *Fluke_8846A_class.py* implements the communication with the DMM Fluke 8846A via LAN interface and SCPI commands using TCP sockets.

In [1]:
# import wrapper class Fluke_8846A from python file Fluke_8846A_class.py
from Fluke_8846A_class import Fluke_8846A

import time

In [2]:
# IP of devices
#dmm_ip = "192.168.10.117"
dmm_ip = "192.168.12.134"

# port for SCPI connection
dmm_port = 3490

# create new device object for the digital multimeter (DMM) Fluke 8846A
dmm = Fluke_8846A(tcp_ip = dmm_ip, tcp_port = dmm_port)

In [22]:
# read connection state of the device
dmm.status

'Disconnected'

In [4]:
# read connection path (at the moment there is only TCP/IP implemented)
dmm.connected_with

'FLUKE 8846A over LAN on 192.168.12.134, port 3490'

In [5]:
# get device information
dmm.getDevInfos()

['FLUKE', '8846A', '2034021', '08/02/10-11:53']

In [21]:
# close the connection to the device
dmm.closeConnection()

In [26]:
# open the connection again
dmm.openConnection(tcp_ip = dmm_ip, tcp_port = dmm_port)

In [6]:
# get a list of valid configurations for temperature measurement
list(dmm.conf_measurement_dict.keys())

['00_RES',
 '01_FRES',
 '02_RTD',
 '03_FRTD',
 '04_RTD_RES',
 '05_FRTD_RES',
 '06_VOLT_AC',
 '07_VOLT_AC_FREQ',
 '08_VOLT_DC',
 '09_CURR_AC',
 '10_CURR_AC_FREQ',
 '11_CURR_DC',
 '12_CONT',
 '13_CAP']

In [63]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('01_FRES')

In [27]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('04_RTD_RES')

In [29]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('08_VOLT_DC')

In [20]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('09_CURR_AC')

In [21]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('12_CONT')

In [7]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('13_CAP')

In [28]:
# get current configuration
dmm.getConfig()

'"TEMP:RTD +0.000000E+00,+1.000000E-03"'

In [30]:
# get measurement with current configuration
dmm.getMeasurement()

{'temperature_value': 26.0047,
 'temperature_unit': '°C',
 'resistance_value': 110.1243,
 'resistance_unit': 'Ohm'}

## Automation in a sample program for measuring temperature

In this sample program, a temperature sensor Pt100 is read out continuously:

In [32]:
import time

# import wrapper class Fluke_8846A from python file Fluke_8846A_class.py
from Fluke_8846A_class import Fluke_8846A

INTERVAL = 0.2

# IP of devices
#dmm_ip = "192.168.10.117"
dmm_ip = "192.168.12.134"

# port for SCPI connection
dmm_port = 3490

# create new device object for the digital multimeter (DMM) Fluke 8846A
dmm = Fluke_8846A(tcp_ip = dmm_ip, tcp_port = dmm_port)

# get device information
dmm.getDevInfos()

# configure DMM for temperature measurement with sensor type Pt100
# with its corresponding resistance value on the secondary display
dmm.confMeasurement('04_RTD_RES')

while True:
    try:
        # get measurement with current configuration
        ret_dict = dmm.getMeasurement()
        
        print("<{:s}> Temperature: {:.3f} {}, Resistance: {:.3f} {}".format(time.strftime('%H:%M:%S'), 
                                                                            ret_dict['temperature_value'], 
                                                                            ret_dict['temperature_unit'],
                                                                            ret_dict['resistance_value'],
                                                                            ret_dict['resistance_unit']))
            
        time.sleep(INTERVAL)
        
    except:
        print("Keyboard Interrupt ^C detected.")
        print("Bye.")
        # close the connection to the device
        dmm.closeConnection()
        break

<21:54:43> Temperature: 25.968 °C, Resistance: 110.110 Ohm
<21:54:44> Temperature: 25.970 °C, Resistance: 110.111 Ohm
<21:54:45> Temperature: 25.970 °C, Resistance: 110.111 Ohm
<21:54:46> Temperature: 25.970 °C, Resistance: 110.111 Ohm
<21:54:47> Temperature: 25.971 °C, Resistance: 110.111 Ohm
<21:54:48> Temperature: 25.969 °C, Resistance: 110.110 Ohm
<21:54:49> Temperature: 25.969 °C, Resistance: 110.110 Ohm
<21:54:50> Temperature: 25.968 °C, Resistance: 110.110 Ohm
<21:54:51> Temperature: 25.969 °C, Resistance: 110.110 Ohm
<21:54:52> Temperature: 25.966 °C, Resistance: 110.109 Ohm
<21:54:53> Temperature: 25.968 °C, Resistance: 110.110 Ohm
<21:54:53> Temperature: 25.967 °C, Resistance: 110.110 Ohm
<21:54:54> Temperature: 25.965 °C, Resistance: 110.109 Ohm
<21:54:55> Temperature: 25.968 °C, Resistance: 110.110 Ohm
<21:54:56> Temperature: 25.967 °C, Resistance: 110.110 Ohm
<21:54:57> Temperature: 25.966 °C, Resistance: 110.109 Ohm
<21:54:58> Temperature: 25.966 °C, Resistance: 110.109 O